In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
import pandas as pd
from account.models import User, InstructorGroup
from booth.models import Participation
from player.models import Player
from django.db.models import Value, IntegerField
from django.db.models.functions import Cast

In [2]:
InstructorGroup.objects.first().get_player()

FieldError: Cannot resolve keyword 'students' into field. Choices are: id, instructor, instructor_group_student, name

In [13]:
player_df =  pd.DataFrame(
            Player.objects \
                .filter(user__user_type='student') \
                .annotate(user_id_=Cast('user__id', IntegerField())) \
                .values('user_id_') \
                .annotate(total_score=F('born_health_score') +
                                    F('born_skill_score') +
                                    F('born_relationship_score') +
                                    F('born_growth_score')
                ).all()
        )
        
parti_df  = pd.DataFrame(
    Participation.objects \
        .filter(player__user__user_type='student') \
        .annotate(user_id_=Cast('player__user__id', IntegerField())) \
        .values('user_id_') 
        .annotate(total_score=
            Sum('score__health_score') +
            Sum('score__skill_score') +
            Sum('score__relationship_score') +
            Sum('score__growth_score')
        )
)
best_score_df = pd.concat([parti_df, player_df]).groupby('user_id_', as_index=False)['total_score'].sum()
best_score_df = best_score_df.rename(columns={'user_id_': 'player'})
best_score_df = best_score_df.sort_values('total_score', ascending=False)

In [14]:
best_score_df


,player,total_score
50,57,979
96,103,968
191,198,910
163,170,887
109,116,874
...,...,...
120,127,140
267,274,135
118,125,125
230,237,120
